# 使用 Pinecone 进行检索增强生成

此笔记本演示了如何通过称为检索增强生成（RAG）的技术将 Claude 与 Pinecone 向量数据库中的数据连接起来。我们将涵盖以下步骤：

1. 使用 Voyage AI 的嵌入模型嵌入数据集
2. 将嵌入上传到 Pinecone 索引
3. 从向量数据库检索信息
4. 使用 Claude 根据数据库中的信息回答问题

## 设置

首先，让我们安装必要的库并设置我们在此笔记本中需要使用的 API 密钥。我们需要获取 [Claude API 密钥](https://docs.claude.com/claude/reference/getting-started-with-the-api)、免费的 [Pinecone API 密钥](https://docs.pinecone.io/docs/quickstart) 和免费的 [Voyage AI API 密钥](https://docs.voyageai.com/install/)。

In [ ]:
%pip install anthropic datasets pinecone-client voyageai

In [ ]:
# 在此处插入您的 API 密钥
ANTHROPIC_API_KEY = "<YOUR_ANTHROPIC_API_KEY>"
PINECONE_API_KEY = "<YOUR_PINECONE_API_KEY>"
VOYAGE_API_KEY = "<YOUR_VOYAGE_API_KEY>"

## 下载数据集

现在让我们下载包含超过 10,000 个亚马逊产品描述的亚马逊产品数据集，并将其加载到 DataFrame 中。

In [ ]:
import pandas as pd

# 下载 JSONL 文件
!wget  https://www-cdn.anthropic.com/48affa556a5af1de657d426bcc1506cdf7e2f68e/amazon-products.jsonl

data = []
with open("amazon-products.jsonl", "r") as file:
    for line in file:
        try:
            data.append(eval(line))
        except Exception:
            pass

df = pd.DataFrame(data)
display(df.head())
len(df)

## 向量数据库

要创建我们的向量数据库，我们首先需要一个来自 Pinecone 的免费 API 密钥。一旦我们有了密钥，我们可以按如下方式初始化数据库：

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

接下来，我们设置索引规范，这使我们能够定义云提供商和我们想要部署索引的区域。您可以在[此处](https://www.pinecone.io/docs/data-types/metadata/)找到所有可用的提供商和区域列表。

In [ ]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(cloud="aws", region="us-west-2")

然后，我们初始化索引。我们将使用 Voyage 的 "voyage-2" 模型来创建嵌入，因此我们将维度设置为 1024。

In [ ]:
import time

index_name = "amazon-products"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

# 检查索引是否已存在（如果这是第一次，应该不存在）
if index_name not in existing_indexes:
    # 如果不存在，创建索引
    pc.create_index(
        index_name,
        dimension=1024,  # voyage-2 嵌入的维度
        metric="dotproduct",
        spec=spec,
    )
    # 等待索引初始化
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

# 连接到索引
index = pc.Index(index_name)
time.sleep(1)
# 查看索引统计信息
index.describe_index_stats()

我们应该看到新的 Pinecone 索引的 total_vector_count 为 0，因为我们还没有添加任何向量。

## 嵌入

要开始使用 Voyage 的嵌入，请前往[此处](https://www.voyageai.com)获取 API 密钥。

现在让我们设置 Voyage 客户端并演示如何使用 `embed` 方法创建嵌入。要了解更多关于在 Claude 中使用 Voyage 嵌入的信息，请参阅[此笔记本](https://github.com/anthropics/anthropic-cookbook/blob/main/third_party/VoyageAI/how_to_create_embeddings.md)。

In [ ]:
import voyageai

vo = voyageai.Client(api_key=VOYAGE_API_KEY)

texts = ["Sample text 1", "Sample text 2"]

result = vo.embed(texts, model="voyage-2", input_type="document")
print(result.embeddings[0])
print(result.embeddings[1])

## 将数据上传到 Pinecone 索引

通过设置嵌入模型，我们现在可以获取产品描述，对它们进行嵌入，并将嵌入上传到 Pinecone 索引。

In [ ]:
from tqdm.auto import tqdm
from time import sleep

descriptions = df["text"].tolist()
batch_size = 100  # 一次创建和插入多少个嵌入

for i in tqdm(range(0, len(descriptions), batch_size)):
    # 找到批次末尾
    i_end = min(len(descriptions), i + batch_size)
    descriptions_batch = descriptions[i:i_end]
    # 创建嵌入（添加 try-except 以避免 RateLimitError。Voyage 目前允许每分钟 300 个请求。）
    done = False
    while not done:
        try:
            res = vo.embed(descriptions_batch, model="voyage-2", input_type="document")
            done = True
        except Exception:
            sleep(5)

    embeds = [record for record in res.embeddings]
    # 为每个文本创建唯一 ID
    ids_batch = [f"description_{idx}" for idx in range(i, i_end)]

    # 为每个文本创建元数据字典
    metadata_batch = [{"description": description} for description in descriptions_batch]

    to_upsert = list(zip(ids_batch, embeds, metadata_batch))

    # 插入到 Pinecone
    index.upsert(vectors=to_upsert)

## 进行查询

填充索引后，我们可以开始进行查询来获取结果。我们可以提出自然语言问题，对其进行嵌入，并针对索引查询以返回语义相似的产品描述。

In [49]:
USER_QUESTION = (
    "I want to get my daughter more interested in science. What kind of gifts should I get her?"
)

question_embed = vo.embed([USER_QUESTION], model="voyage-2", input_type="query")
results = index.query(vector=question_embed.embeddings, top_k=5, include_metadata=True)
results

{'matches': [{'id': 'description_1771',
              'metadata': {'description': 'Product Name: Scientific Explorer '
                                          'My First Science Kids Science '
                                          'Experiment Kit\n'
                                          '\n'
                                          'About Product: Experiments to spark '
                                          'creativity and curiosity | Grow '
                                          'watery crystals, create a rainbow '
                                          'in a plate, explore the science of '
                                          'color and more | Represents STEM '
                                          '(Science, Technology, Engineering, '
                                          'Math) principles – open ended toys '
                                          'to construct, engineer, explorer '
                                          'and experiment | Inclu

## 优化搜索

这些结果不错，但我们可以进一步优化它们。使用 Claude，我们可以获取用户的问题并从中生成搜索关键词。这使我们能够对索引执行广泛、多样的搜索，以获得更相关的产品描述。

In [ ]:
import anthropic

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)


def get_completion(prompt):
    completion = client.completions.create(
        model="claude-2.1",
        prompt=prompt,
        max_tokens_to_sample=1024,
    )
    return completion.completion

In [ ]:
def create_keyword_prompt(question):
    return f"""\n\nHuman: Given a question, generate a list of 5 very diverse search keywords that can be used to search for products on Amazon.

The question is: {question}

Output your keywords as a JSON that has one property "keywords" that is a list of strings. Only output valid JSON.\n\nAssistant:{{"""

通过我们的 Anthropic 客户端设置和创建的提示，我们现在可以开始从问题生成关键词。我们将在 JSON 对象中输出关键词，以便我们可以轻松地从 Claude 的输出中解析它们。

In [ ]:
keyword_json = "{" + get_completion(create_keyword_prompt(USER_QUESTION))
print(keyword_json)

In [ ]:
import json

# Extract the keywords from the JSON
data = json.loads(keyword_json)
keywords_list = data["keywords"]
print(keywords_list)

现在我们有了列表中的关键词，让我们嵌入每一个，针对索引查询，并返回前 3 个最相关的产品描述。

In [ ]:
results_list = []
for keyword in keywords_list:
    # get the embeddings for the keywords
    query_embed = vo.embed([keyword], model="voyage-2", input_type="query")
    # search for the embeddings in the Pinecone index
    search_results = index.query(vector=query_embed.embeddings, top_k=3, include_metadata=True)
    # append the search results to the list
    for search_result in search_results.matches:
        results_list.append(search_result["metadata"]["description"])
print(len(results_list))

## 用 Claude 回答

现在我们有了产品描述列表，让我们将它们格式化为 Claude 训练过的搜索模板，并将格式化的描述传递到另一个提示中。

In [ ]:
# 格式化搜索结果
def format_results(extracted: list[str]) -> str:
    result = "\n".join(
        [
            f'<item index="{i + 1}">\n<page_content>\n{r}\n</page_content>\n</item>'
            for i, r in enumerate(extracted)
        ]
    )

    return f"\n<search_results>\n{result}\n</search_results>"


def create_answer_prompt(results_list, question):
    return f"""\n\nHuman: {format_results(results_list)} 使用 <search_results></search_results> 标签内提供的搜索结果，请回答以下问题 <question>{question}</question>。不要在您的回答中引用搜索结果。\n\nAssistant:"""

最后，让我们询问原始用户的问题并从 Claude 那里获得我们的答案。

In [50]:
answer = get_completion(create_answer_prompt(results_list, USER_QUESTION))
print(answer)

 To get your daughter more interested in science, I would recommend getting her an age-appropriate science kit or set that allows for hands-on exploration and experimentation. For example, for a younger child you could try a beginner chemistry set, magnet set, or crystal growing kit. For an older child, look for kits that tackle more advanced scientific principles like physics, engineering, robotics, etc. The key is choosing something that sparks her natural curiosity and lets her actively investigate concepts through activities, observations, and discovery. Supplement the kits with science books, museum visits, documentaries, and conversations about science she encounters in everyday life. Making science fun and engaging is crucial for building her interest.
